# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [ ]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

## District Summary

In [ ]:
# Calculate the total number of unique schools
school_count = len(school_data_complete.school_name.unique())
school_count

In [ ]:
# Calculate the total number of students
student_count = school_data_complete.student_name.count()
student_count

In [ ]:
# Calculate the total budget
total_budget = sum(school_data_complete.budget.unique())
total_budget

In [ ]:
# Calculate the average (mean) math score
average_math_score = school_data_complete.math_score.mean()
average_math_score

In [ ]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete.reading_score.mean()
average_reading_score

In [ ]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

In [ ]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

In [ ]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

In [ ]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({"Total Schools":[school_count],
                                 "Total Students":[student_count],
                                 "Total Budget":[total_budget],
                                 "Average Math Score":[average_math_score],
                                 "Average Reading Score":[average_reading_score],
                                 "% Passing Math":[passing_math_percentage*100],
                                 "% Passing Reading":[passing_reading_percentage*100],
                                 "% Overall Passing":[overall_passing_rate*100]})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

## School Summary

In [ ]:
# Use the code provided to select the school type
school_types = school_data.set_index(["school_name"])["type"]
school_data

In [ ]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
# Grouped our complete data frame by school name 
grouped_school = school_data_complete.groupby(['school_name'])
# Calculate the total student for each school
total_student = grouped_school.size()
# Get the school type for each school
school_type = grouped_school['type'].first()
# Calculate the total budget for each school
total_budget = grouped_school['budget'].first()
# Calculate the budget per student for each school
t_budget_per_student = total_budget/total_student
# Calculate the average math score for each school
average_math_score = grouped_school['math_score'].mean()
# Calculate the average reading score for each school
average_reading_score = grouped_school['reading_score'].mean()
# Calculate the percentange of passing math score for each school
grouped_passing_math = school_data_complete[school_data_complete['math_score']>=70].groupby(['school_name']).size()
percent_passing_math = (grouped_passing_math/total_student)*100
# Calculate the percentange of passing math score for each school
grouped_passing_reading = school_data_complete[school_data_complete['reading_score']>=70].groupby(['school_name']).size()
percent_passing_reading = (grouped_passing_reading/total_student)*100
# Calculate the overall passing score for each school
percent_overall_passing = (percent_passing_math + percent_passing_reading)/2

# Create a dataframe to hold the above results
school={
    'School Type': school_type,
    'Total Students':total_student,
    'Total School Budget': total_budget,
    'Per Student Budget': t_budget_per_student,
    'Average Math Score': average_math_score,
    'Average Reading Score': average_reading_score,
    '% Passing Math': percent_passing_math,
    '% Passing Reading':percent_passing_reading,
    '% Overall Passing Rate': percent_overall_passing,
}
per_school_summary = pd.DataFrame(school)

# Create a copy of school summary data frame before formatting to be able to use the numeric data on original data frame later
displayed_school_summary = per_school_summary.copy()
# Formatting the display data frame
displayed_school_summary['Per Student Budget'] = displayed_school_summary['Per Student Budget'].map('${:,.2f}'.format)
displayed_school_summary['Total School Budget'] = displayed_school_summary['Total School Budget'].map('${:,.2f}'.format)
displayed_school_summary.index.name = None
# Display the DataFrame
displayed_school_summary

## Highest-Performing Schools (by % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = displayed_school_summary.sort_values(by='% Overall Passing Rate', ascending=False)
top_schools.head(5)

## Bottom Performing Schools (By % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = displayed_school_summary.sort_values(by='% Overall Passing Rate', ascending=True)
bottom_schools.head(5)

## Math Scores by Grade

In [94]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_grade_math_scores = ninth_graders.groupby('school_name')['math_score'].mean()
tenth_grader_math_scores = tenth_graders.groupby('school_name')['math_score'].mean()
eleventh_grader_math_scores = eleventh_graders.groupby('school_name')['math_score'].mean()
twelfth_grader_math_scores = twelfth_graders.groupby('school_name')['math_score'].mean()


# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
grade_reading_score={
    '9th':ninth_grade_math_scores,
    '10th':tenth_grader_math_scores,
    '11th':eleventh_grader_math_scores,
    '12th':twelfth_grader_math_scores,
    }
math_score_by_grade= pd.DataFrame(grade_reading_score)
# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade.head()

AttributeError: 'dict' object has no attribute 'index'

## Reading Score by Grade 

In [92]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_grade_reading_scores = ninth_graders.groupby('school_name')['reading_score'].mean()
tenth_grader_reading_scores = tenth_graders.groupby('school_name')['reading_score'].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby('school_name')['reading_score'].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby('school_name')['reading_score'].mean()


# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
grade_reading_score={
    '9th':ninth_grade_reading_scores,
    '10th':tenth_grader_reading_scores,
    '11th':eleventh_grader_reading_scores,
    '12th':twelfth_graders_scores,
    }

reading_score_by_grade= pd.DataFrame(grade_reading_score)

# Minor data wrangling

reading_score_by_grade.index.name = None

# Display the DataFrame
reading_score_by_grade.head(5)

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


## Scores by School Spending

In [95]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [96]:
# Create a copy of the school summary since it has the "Per Student Budget" 
#school_spending_df = per_school_summary.copy()

In [104]:
# Use `pd.cut` to categorize spending based on the bins.

school_spending_df = per_school_summary.loc[:,['Average Math Score',
                                  'Average Reading Score','% Passing Math',
                                  '% Passing Reading','% Overall Passing Rate',]]
# Add a new columns named Spending Ranges (Per Student) and binning based off budget per student
school_spending_df['Spending Ranges (Per Student)']= pd.cut(per_school_summary['Per Student Budget'],spending_bins,labels=labels)
# Create a group based off of the bins
school_spending_df = school_spending_df.groupby('Spending Ranges (Per Student)').mean()
school_spending_df.head()

C:\Users\sthotad\AppData\Local\Temp\ipykernel_9632\4215861479.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  school_spending_df = school_spending_df.groupby('Spending Ranges (Per Student)').mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-630,81.899826,83.155286,87.133538,92.718205,89.925871
$630-645,78.518855,81.624473,73.484209,84.391793,78.938001
$645-680,76.997210,81.027843,66.164813,81.133951,73.649382


In [106]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing Rate"]

C:\Users\sthotad\AppData\Local\Temp\ipykernel_9632\415376001.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
C:\Users\sthotad\AppData\Local\Temp\ipykernel_9632\415376001.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
C:\Users\sthotad\AppData\Local\Temp\ipykernel_9632\415376001.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in

In [107]:
# Assemble into DataFrame
spending_summary = 

# Display results
spending_summary

SyntaxError: invalid syntax (1784604702.py, line 2)

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 
per_school_summary

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = 

# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = 
type_reading_scores = 
type_passing_math = 
type_passing_reading = 
type_overall_passing = 

# Use the code provided to select new column data
average_math_score_by_type = type_math_scores["Average Math Score"]
average_reading_score_by_type = type_reading_scores["Average Reading Score"]
average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing"]

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = 

# Display results
type_summary